In [2]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from MyModule import myhandlandmark as ml

In [6]:
midas = torch.hub.load('intel-isl/MiDaS', 'DPT_Hybrid')
midas.eval()
device = torch.device('cuda')
midas.to(device)

Using cache found in C:\Users\user/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\user\anaconda3\envs\cv_env\lib\site-packages\timm\models\_factory.py:126: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): HybridEmbed(
        (backbone): ResNetV2(
          (stem): Sequential(
            (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
            (norm): GroupNormAct(
              32, 64, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inplace=True)
            )
            (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
          )
          (stages): Sequential(
            (0): ResNetStage(
              (blocks): Sequential(
                (0): Bottleneck(
                  (downsample): DownsampleConv(
                    (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                    (norm): GroupNormAct(
                      32, 256, eps=1e-05, affine=True
                      (drop): Identity()
                      (act): Identit

In [7]:
midas_transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = midas_transforms.dpt_transform

Using cache found in C:\Users\user/.cache\torch\hub\intel-isl_MiDaS_master


In [ ]:
hand = ml.handtracking()
    
# 비디오 파일 로드
cap = cv2.VideoCapture(1)
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img = hand.preprogress(frame)
    detection_result = hand.detector.detect(img)
    annotated_img, width, height = hand.draw_landmarks_on_image(img.numpy_view(), detection_result)
    hand_landmarks_list = detection_result.hand_landmarks
    
    
    input_batch = transform(img.numpy_view()).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size = (480,640),
            mode = 'bicubic',
            align_corners = False
        ).squeeze()
    depth_map = prediction.cpu().numpy()

    depth_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    depth_colored = cv2.applyColorMap(depth_normalized, cv2.COLORMAP_JET)
    
    if len(hand_landmarks_list) > 0:
        for i, hand_landmark in enumerate(hand_landmarks_list):
            h, w = 480, 640
            x = int(hand_landmark[0].x * w)
            y = int(hand_landmark[0].y * h)

            if 0 <= x < w and 0 <= y < h:
                depth = depth_map[y, x]
                # print(f"[{i}] Landmark 0 depth: {depth:.2f}")

                cv2.putText(annotated_img, f"Depth : {depth:.2f}", 
                            (45, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # # 4. 결과 출력
    cv2.imshow('Hand + Depth', cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR))
    cv2.imshow('DepthMap', depth_normalized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[[NormalizedLandmark(x=0.034227821975946426, y=0.20834949612617493, z=-2.293661509611411e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.06598563492298126, y=0.19155512750148773, z=-0.0007979335496202111, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.09560166299343109, y=0.19707104563713074, z=-0.0009351748158223927, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.11438775062561035, y=0.2167816311120987, z=-0.0011976079549640417, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.12428859621286392, y=0.23200224339962006, z=-0.00043452606769278646, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.09757388383150101, y=0.23282933235168457, z=-0.006082953419536352, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.12506449222564697, y=0.25542452931404114, z=-0.007861939258873463, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.1317482441663742, y=0.24789349734783173, z=-0.006656612269580364, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.12